# Graph Neural Networks Baseline
with diffrent node features strategies
 
 - timeseries
 - correlations

In [ ]:
import numpy as np
import pickle
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

In [ ]:
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from sklearn.model_selection import StratifiedKFold

## Load data

In [ ]:
DATA_FOLDER = '../data'
PICKLE_FOLDER = '../pickles'

In [ ]:
with open(f'{PICKLE_FOLDER}/timeseries.pickle', 'rb') as f:
    ts = pickle.load(f)

In [ ]:
total_samples, total_brain_regions, ts_length = ts.shape

In [ ]:
df_metadata = pd.read_csv(f'{DATA_FOLDER}/patients-cleaned.csv', index_col=0)

In [ ]:
df_metadata.head(2)

### Select connectivity dataset

In [ ]:
THRESHOLD = 0.1                         # 0.01, 0.05, 0.1, 0.15
N = 10 #                                # 3, 5, 7, 10, 15, 20, 40
EDGE_FEATURES = 'binary'                # 'binary', 'real'
CORR_TYPE = 'spearman'                  # 'pearson', 'spearman', 'partial-pearson'
THRESHOLD_METHOD = 'abs-sample-diff'    # 'abs-sample-diff', 'abs-group-avg-diff'
THRESHOLD_TYPE = 'large'                # 'min', 'max' or for kNN 'small', 'large'
KNN = True                              # Whether all or only top N neigbors are taken

In [ ]:
fc_folder = f'{PICKLE_FOLDER}/fc-{CORR_TYPE}{"-knn-" if KNN else ""}{THRESHOLD_METHOD}'

In [ ]:
fc_file = f'{fc_folder}/{THRESHOLD_TYPE}-{f"knn-{N}" if KNN else f"th-{THRESHOLD}"}-{EDGE_FEATURES}.pickle'

In [ ]:
with open(fc_file, 'rb') as f:
    edge_index_matrix = pickle.load(f)

In [ ]:
edge_index_matrix.shape

## Split data

In [ ]:
with open(f'{PICKLE_FOLDER}/test-indices.pickle', 'rb') as f:
    test_indices = pickle.load(f)

In [ ]:
train_indices = list(set(range(total_samples)) - set(test_indices))

In [ ]:
train_targets = df_metadata.iloc[train_indices]["target"].reset_index(drop=True)

In [ ]:
print(f'Train set size: {len(train_indices)}')
print(f'Test set size: {len(test_indices)}')

## Extend dataset

In [ ]:
N_SURROGATES = 5                    # 5
SURROGATE_METHOD = 'iaaft'          # 'iaaft', 'aaft'

In [ ]:
with open(f'{PICKLE_FOLDER}/timeseries-{SURROGATE_METHOD}-{N_SURROGATES}.pickle', 'rb') as f:
    ts_surrogates = pickle.load(f)

In [ ]:
print(f'Extra data: {ts_surrogates.shape}')
print(f'{ts_surrogates.shape[0] / total_samples} surrogates per sample')

Extend only training data. Test set will consist of true data only.

In [ ]:
ts_train_surrogates = np.concatenate([ts_surrogates[N_SURROGATES*i:N_SURROGATES*i+N_SURROGATES] for i in train_indices], axis=0)

In [ ]:
ts_train_surrogates.shape

## Prepare data

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

### `Data` object fields

- `data.x`: Node feature matrix with shape `[num_nodes, num_node_features]`

- `data.edge_index`: Graph connectivity in COO format with shape `[2, num_edges]` and type `torch.long`

- `data.edge_attr`: Edge feature matrix with shape `[num_edges, num_edge_features]`

- `data.y`: Target to train against (may have arbitrary shape), e.g., node-level targets of shape `[num_nodes, *]` or graph-level targets of shape `[1, *]`

- `data.pos`: Node position matrix with shape `[num_nodes, num_dimensions]`

### Select node features

- timeseries
- correlations

In [ ]:
ts_length_start = 50
ts_length_end = ts_length

In [ ]:
dataset = [Data(
    x=torch.from_numpy(ts[i][:,ts_length_start:ts_length_end]).to(torch.float32),  
    edge_index=torch.from_numpy(np.asarray(np.nonzero(edge_index_matrix[i]))).to(torch.int64),
    y=torch.tensor([target], dtype=torch.int64)
).to(device) for target, i in zip(train_targets, train_indices)]

In [ ]:
print(f'True train data: {len(dataset)}')

In [ ]:
# Extension.
dataset_ext = [Data(
    x=torch.from_numpy(ts_surr[:,ts_length_start:ts_length_end]).to(torch.float32),  
    edge_index=torch.from_numpy(np.asarray(np.nonzero(edge_index_matrix[i]))).to(torch.int64),
    y=torch.tensor([target], dtype=torch.int64)
).to(device) for target, i, ts_surr in zip(
    np.repeat(train_targets, N_SURROGATES),
    np.repeat(train_indices, N_SURROGATES),
    ts_train_surrogates
)]

In [ ]:
print(f'Surrogate train data: {len(dataset_ext)}')

In [ ]:
dataset.extend(dataset_ext)

In [ ]:
print(f'Extended train data: {len(dataset)}')

In [ ]:
print('Data object')
print(f'Edge index: {dataset[0].edge_index.shape}')
print(f'Node features: {dataset[0].x.shape}')
print(f'Target: {dataset[0].y.shape}')

In [ ]:
train_targets_ext = np.concatenate([train_targets, np.repeat(train_targets, N_SURROGATES)])
train_targets_ext.shape

## Define model

In [ ]:
class BaselineGCN(torch.nn.Module):
    
    def __init__(self, hidden_channels):
        super(BaselineGCN, self).__init__()
        torch.manual_seed(42)
        self.conv1 = GCNConv(ts_length_end-ts_length_start, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)

        self.fc1 = Linear(hidden_channels, 2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # 1. Obtain node embeddings 
        x = F.gelu(self.conv1(x, edge_index))
        x = F.gelu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc1(x)     # CELoss already incorporates `log_softmax`.
        
        return x

In [ ]:
# Architecture.
BaselineGCN(hidden_channels=8)

## Evaluation

In [ ]:
def evaluation_metrics(predicted, labels):
    pred_positives = predicted == 1
    label_positives = labels == 1

    tp = (pred_positives & label_positives).sum().item()
    tn = (~pred_positives & ~label_positives).sum().item()
    fp = (pred_positives & ~label_positives).sum().item()
    fn = (~pred_positives & label_positives).sum().item()

    return tp, tn, fp, fn

## Train model

In [ ]:
NUM_FOLDS = 3

In [ ]:
skf = StratifiedKFold(n_splits=NUM_FOLDS, random_state=42, shuffle=True)

In [ ]:
# Settings.
EPOCHS = 150
LR = 0.001
MOMENTUM = 0.9
OPTIMIZER = 'sgd'
LOSS = 'ce'
BATCH_SIZE = 2

VALIDATE_FREQ = 10

HIDDEN_CHANNELS = 32

In [ ]:
# Experiment folder.
EXP_FOLDER = 'runs/node-features'

In [ ]:
# Experiment.
EXP_ID = 7

In [ ]:
for kfold, (train_index, val_index) in enumerate(skf.split(np.zeros(len(train_targets_ext)), train_targets_ext)):

    # Init TB writer.
    foldername = f'id={EXP_ID:03d},fold={kfold},bs={BATCH_SIZE},e={EPOCHS},lr={LR},mom={MOMENTUM},opt={OPTIMIZER},loss={LOSS}'
    writer = SummaryWriter(f"../{EXP_FOLDER}/{foldername}")

    # Init model.
    net = BaselineGCN(hidden_channels=HIDDEN_CHANNELS).to(device)
    optimizr = torch.optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM)
    criterion = torch.nn.CrossEntropyLoss()

    # Save architecture.
    with open(f"../{EXP_FOLDER}/{foldername}/architecture", 'w') as f:
        f.write(fc_file)
        f.write('\n\n')
        f.write(net.__str__())

    # Prepare data.
    X_train = [dataset[i] for i in train_index]
    X_val = [dataset[i] for i in val_index]
    
    trainloader = DataLoader(X_train, batch_size=BATCH_SIZE, shuffle=True)
    valloader = DataLoader(X_val, batch_size=BATCH_SIZE, shuffle=False)

    # Train.
    for epoch in range(EPOCHS):
        running_loss = 0.
        
        for data in trainloader:
            net.train()
            optimizr.zero_grad()

            outputs = net(data)
            
            loss = criterion(outputs, data.y)  # Compute the loss.
            loss.backward()  # Derive gradients.
            optimizr.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(trainloader)
        writer.add_scalar('training loss', epoch_loss, epoch)

        running_loss = 0.

        # Evaluate epoch.
        tp, tn, fp, fn = 0, 0, 0, 0
        total = 0

        for data in valloader:
            net.eval()
            optimizr.zero_grad()

            outputs = net(data)
            
            loss = criterion(outputs, data.y)  # Compute the loss.
            running_loss += loss.item()

            if (epoch+1) % VALIDATE_FREQ == 0:
                predicted = outputs.argmax(dim=1)
                labels = data.y.view(-1)

                # Update.
                tp_, tn_, fp_, fn_ = evaluation_metrics(predicted, labels)
                tp += tp_; tn += tn_; fp += fp_; fn += fn_
                total += data.y.size(0)

        epoch_loss = running_loss / len(valloader)
        writer.add_scalar('validation loss', epoch_loss, epoch)
        
        if (epoch+1) % VALIDATE_FREQ == 0:
            writer.add_scalar('validation accuracy', (tp + tn) / total, epoch)
            writer.add_scalar('validation precision', tp / (tp + fp), epoch)
            writer.add_scalar('validation recall', tp / (tp + fn), epoch)
    
    # Single fold during exploration.
    break

print('Finished training')